In [1]:
import numpy as np
import pandas as pd
import jax
import jax.numpy as jnp
from jax import random
from flax.training import train_state
from jax.experimental.ode import odeint

import time

# RELOADING MODULES
import importlib
import neural_net_pyomo
import matplotlib.pyplot as plt

importlib.reload(neural_net_pyomo)
NeuralODEPyomo = neural_net_pyomo.NeuralODEPyomo

# UTILS DIRECTORY
import sys
import os
collocation2_path = os.path.abspath(os.path.join('..', 'utils'))

if collocation2_path not in sys.path:
    sys.path.append(collocation2_path)

from collocation import compute_weights, lagrange_derivative

import non_parametric_collocation 
importlib.reload(non_parametric_collocation)
collocate_data = non_parametric_collocation.collocate_data

import data_generation 
importlib.reload(data_generation)
generate_ode_data = data_generation.generate_ode_data

In [30]:
t_train = np.squeeze(np.array(pd.read_csv('../data/daily_climate/train/X_train.csv'), dtype=jnp.float32)) # x is timepoints t
t_test = np.squeeze(np.array(pd.read_csv('../data/daily_climate/test/X_test.csv'), dtype=jnp.float32))

Y_train = np.squeeze(np.array(pd.read_csv('../data/daily_climate/train/y_train.csv'))) 
Y_test = np.squeeze(np.array(pd.read_csv('../data/daily_climate/test/y_test.csv'))) # y is the mean temperature

In [31]:
t_train

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.],
      dtype=float32)

In [33]:
estimated_derivative, estimated_solution = collocate_data(Y_train, t_train, 'EpanechnikovKernel', bandwidth=2)
estimated_solution = np.array(estimated_solution)

/Users/mariiashapo/Library/CloudStorage/OneDrive-Personal/project_2324/NODE/utils/non_parametric_collocation.py:95: UserWarning: Data transposed to match expected dimensions.
  warnings.warn("Data transposed to match expected dimensions.")


In [34]:
weights = compute_weights(t_train)
D = np.array(lagrange_derivative(t_train, weights))

In [35]:
print(Y_train.shape)
print(t_train.shape)
print(estimated_solution.shape)

(26, 4)
(26,)
(4, 26)


In [36]:
layer_widths = [5, 20, 20, 4]

# solver.options['print_level'] = 5
params = {"tol":1e-7, "print_level": 5}

start_timer_build = time.time()
ode_model = NeuralODEPyomo(Y_train, t_train, D, layer_widths, max_iter = 500, act_func = "tanh", y_init = estimated_solution.T, extra_input = t_train, params = params) # , extra_input = t_ho
ode_model.build_model()
end_timer_build = time.time()

result = ode_model.solve_model()
u_model = ode_model.extract_solution().T

Ipopt 3.14.16: max_iter=500
halt_on_ampl_error=yes
tol=1e-07
print_level=5


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.1.

Number of nonzeros in equality constraint Jacobian...:    59000
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:   135814

Total number of variables............................:      728
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      104
                     variables with only upper bounds:        0
Total number of 

In [ ]:
def system_dynamics(y, t):
    input = jnp.array([y[0], t])
    du_dt = ode_model.predict(input)
    return jnp.array([du_dt])

y_pred_train = odeint(system_dynamics, Y_train[0], t_train)    
mae_train = jnp.mean(jnp.abs(y_pred_train - Y_train))
print (f"Train MAE: {mae_train}")

y1 = y_pred_train[-1, :]
y_pred_test = odeint(system_dynamics, y1, t_test[N-1:])
mae_test = jnp.mean(jnp.abs(y_pred_test - y_test[N-1:]))
print (f"Test MAE: {mae_test}")

#---------------------------PLOTTING RESULTS--------------------------#
plt.figure(figsize=(10, 5))
plt.plot(t, y_pred_train, 'b', label='Predicted Solution')
plt.plot(t_test[N-1:], y_pred_test, 'b--', label='Predicted Solution', omega = 0.5)

#plt.plot(t_ho, v_model, 'r', label='Predicted Solution')
plt.plot(t_test, y_test, 'g--', label='True Solution')

plt.scatter(t, y_noisy.T, label='Noisy Data u(t)', alpha = 0.5)
plt.xlabel('Time')
plt.ylabel('u(t)')
plt.axvline(x=end_time, color='r', linestyle='--', label='Train-test split')
plt.legend(loc ="lower right")
plt.grid(True)
plt.show()